In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, Dense, Dropout, SimpleRNN, RepeatVector
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback
from termcolor import colored

In [3]:
all_chars='0123456789+' #Vocabulary
num_features = len(all_chars)
print('no of features:', num_features)
char_to_index= dict((c,i) for i,c in enumerate(all_chars))
index_to_char= dict((i,c) for i, c in enumerate(all_chars))

no of features: 11


In [5]:
def generate_data():
    first = np.random.randint(0,100)
    second = np.random.randint(0,100)
    example = str(first)+ '+' + str(second)
    label = str(first+second)
    return example, label
generate_data()

('48+81', '129')

In [7]:
from keras.models import Sequential
from keras.layers import SimpleRNN, RepeatVector, TimeDistributed, Dense, Input

hidden_units = 128
max_time_steps = 5
num_features = 11  # You may need to define num_features based on your data

model = Sequential([
    Input(shape=(None, num_features)),  # Explicit Input layer
    SimpleRNN(hidden_units),
    RepeatVector(max_time_steps),
    SimpleRNN(hidden_units, return_sequences=True),
    TimeDistributed(Dense(num_features, activation='softmax'))
])

model.compile(
   loss='categorical_crossentropy',
   optimizer='adam',
   metrics=['accuracy']
)

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)          │ (None, 128)            │        17,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ repeat_vector (RepeatVector)    │ (None, 5, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ (None, 5, 128)         │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 5, 11)          │         1,419 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 52,235 (204.04 KB)

 Trainable params: 52,235 (204.04 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
def vectorize_example(example,label):
    x=np.zeros((max_time_steps,num_features))
    y=np.zeros((max_time_steps,num_features))
    
    diff_x = max_time_steps - len(example)
    diff_y = max_time_steps - len(label)
    
    for i,c in enumerate(example):
        x[i+diff_x,char_to_index[c]] =1
    for i in range(diff_x):
        x[i,char_to_index['0']] = 1
    for i,c in enumerate(label):
        y[i+diff_y,char_to_index[c]] =1
    for i in range(diff_y):
        y[i,char_to_index['0']] = 1  
    return x,y
e, l = generate_data()
print(e,l)
x,y= vectorize_example(e,l)
print(x.shape,y.shape)

82+77 159
(5, 11) (5, 11)


In [11]:
def devectorize_example(example):
    result = [index_to_char[np.argmax(vec)] for i,vec in enumerate(example)]
    return ''.join(result)
devectorize_example(x)

'82+77'

In [13]:
devectorize_example(y)

'00159'

In [15]:
def create_dataset(num_examples=2000):
    x=np.zeros((num_examples,max_time_steps,num_features))
    y=np.zeros((num_examples,max_time_steps,num_features))
    for i in range(num_examples):
        e,l = generate_data()
        e_v, l_v = vectorize_example(e,l)
        x[i] = e_v
        y[i] = l_v
    return x,y
x,y = create_dataset()
print(x.shape,y.shape)

(2000, 5, 11) (2000, 5, 11)


In [17]:
devectorize_example(x[0])
devectorize_example(y[0])

'00111'

In [ ]:
from keras.callbacks import LambdaCallback, EarlyStopping

l_cb = LambdaCallback(
    on_epoch_end=lambda e, l: print('{:.2f}'.format(l.get('val_accuracy', 0)), end=' _ ')
)
es_cb = EarlyStopping(monitor='val_loss', patience=10)

model.fit(x, y, epochs=500, batch_size=256, validation_split=0.2,
          verbose=False, callbacks=[es_cb, l_cb])


0.57 _ 0.59 _ 0.61 _ 0.62 _ 0.61 _ 0.63 _ 0.63 _ 0.64 _ 0.64 _ 0.64 _ 0.65 _ 0.65 _ 0.66 _ 0.66 _ 0.67 _ 0.68 _ 0.67 _ 0.67 _ 0.70 _ 0.70 _ 0.69 _ 0.70 _ 0.70 _ 0.71 _ 0.70 _ 0.71 _ 0.72 _ 0.73 _ 0.72 _ 0.73 _ 0.74 _ 0.74 _ 0.74 _ 0.75 _ 0.75 _ 0.75 _ 0.75 _ 0.75 _ 0.75 _ 0.75 _ 0.76 _ 0.76 _ 0.77 _ 0.77 _ 0.78 _ 0.77 _ 0.77 _ 0.78 _ 0.79 _ 0.79 _ 0.79 _ 0.80 _ 0.81 _ 0.81 _ 0.82 _ 0.82 _ 0.83 _ 0.83 _ 0.85 _ 0.84 _ 0.84 _ 0.85 _ 0.86 _ 0.85 _ 0.86 _ 0.86 _ 0.87 _ 0.86 _ 0.88 _ 0.88 _ 0.88 _ 0.88 _ 0.89 _ 0.89 _ 0.90 _ 0.90 _ 0.90 _ 0.90 _ 0.91 _ 0.90 _ 0.90 _ 0.91 _ 0.91 _ 0.91 _ 0.92 _ 0.92 _ 0.92 _ 0.92 _ 0.92 _ 0.92 _ 0.93 _ 0.92 _ 0.93 _ 0.93 _ 0.93 _ 0.93 _ 0.94 _ 0.93 _ 0.94 _ 0.93 _ 0.94 _ 0.94 _ 0.93 _ 0.94 _ 0.94 _ 0.93 _ 0.95 _ 0.94 _ 0.94 _ 0.94 _ 0.94 _ 0.94 _ 0.94 _ 0.94 _ 0.94 _ 0.94 _ 0.95 _ 0.94 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.96 _ 0.95 _ 0.95 _ 0.95 _ 0.95 _ 0.96 _ 0.95 _ 0.96 _ 0.96 _

In [30]:
x_test,y_test = create_dataset(10)
preds = model.predict(x_test)
for i,pred in enumerate(preds):
    y=devectorize_example(y_test[i])
    y_hat = devectorize_example(pred)
    col='green'
    if y!= y_hat:
        col='red'
    out='Input: '+ devectorize_example(x_test[i])+' Out: ' +y+'Pred:' +y_hat
    print(colored(out,col))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 342ms/step
Input: 88+76 Out: 00164Pred:00164
Input: 93+79 Out: 00172Pred:00172
Input: 72+23 Out: 00095Pred:00095
Input: 039+5 Out: 00044Pred:00044
Input: 37+92 Out: 00129Pred:00129
Input: 30+49 Out: 00079Pred:00079
Input: 35+80 Out: 00115Pred:00115
Input: 83+35 Out: 00118Pred:00118
Input: 37+49 Out: 00086Pred:00086
Input: 62+57 Out: 00119Pred:00120
